In [15]:
import keras
import numpy as np

import pynucastro as pyna

import chugunov_indicator as chug

fixed_rng = np.random.default_rng(0)

In [10]:
kwargs = {"size": 3*10**5, "rng": fixed_rng}

train = chug.ScreeningFactorData(**kwargs)
validate = chug.ScreeningFactorData(**kwargs)
test = chug.ScreeningFactorData(**kwargs)

network = chug.ScreeningFactorNetwork(train, validate, test, 0)

In [11]:
network.class_weight = {0: 64983, 1: 35017*2}
network.compile(learning_rate=1e-4)

In [12]:
history = network.fit_model(epochs=500, verbose=2)
network.score

Epoch 1/500
147/147 - 4s - 30ms/step - fn: 14425.0000 - fp: 14119.0000 - loss: 22391.5352 - precision: 0.8648 - recall: 0.8623 - tn: 181152.0000 - tp: 90304.0000 - val_fn: 4829.0000 - val_fp: 6016.0000 - val_loss: 0.1021 - val_precision: 0.9433 - val_recall: 0.9540 - val_tn: 188981.0000 - val_tp: 100174.0000
Epoch 2/500
147/147 - 2s - 12ms/step - fn: 4296.0000 - fp: 6064.0000 - loss: 5822.9468 - precision: 0.9431 - recall: 0.9590 - tn: 189207.0000 - tp: 100433.0000 - val_fn: 3411.0000 - val_fp: 4216.0000 - val_loss: 0.0637 - val_precision: 0.9602 - val_recall: 0.9675 - val_tn: 190781.0000 - val_tp: 101592.0000
Epoch 3/500
147/147 - 2s - 11ms/step - fn: 3092.0000 - fp: 4341.0000 - loss: 4093.6570 - precision: 0.9590 - recall: 0.9705 - tn: 190930.0000 - tp: 101637.0000 - val_fn: 2684.0000 - val_fp: 2966.0000 - val_loss: 0.0481 - val_precision: 0.9718 - val_recall: 0.9744 - val_tn: 192031.0000 - val_tp: 102319.0000
Epoch 4/500
147/147 - 2s - 11ms/step - fn: 2490.0000 - fp: 3357.0000 - los

[0.0060164183378219604,
 497.0,
 196.0,
 0.9981310963630676,
 0.9952744841575623,
 194630.0,
 104677.0]

In [ ]:
network.model.save("chugunov_indicator/keras/model.keras")
#network.model = keras.saving.load_model("chugunov_indicator/keras/model.keras")

In [16]:
nuclei = pyna.Nucleus.cast_list(["h1", "he4", "c12", "o16", "n14", "ca40"])
reactants = pyna.Nucleus.cast_list(["c12", "he4"])

comp = pyna.Composition(nuclei=nuclei)
comp.set_solar_like()

scn_fac = pyna.make_screen_factors(*reactants)

In [73]:
dens, temp = 1.e6, 3e8
plasma = pyna.make_plasma_state(temp, dens, comp.get_molar())

In [74]:
pyna.screening.chugunov_2009(plasma, scn_fac)

1.3547275859649746

In [75]:
network.predict_states(plasma, scn_fac)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


True